In [23]:
import os, sys
sys.path.append('..')
sys.path.append('../../')

import torch
import pandas as pd
import torch.nn.functional as F
import matplotlib.pyplot as plt
import glob
import numpy as np
from itertools import chain

import config as conf

In [7]:
read_file_lst = glob.glob(conf.data_root + 'read/*')
exclude_file_lst = ['read.tar']
read_df_lst = []
for f in read_file_lst:
    file_name = os.path.basename(f)
    if file_name in exclude_file_lst:
        print(file_name)
    else:
        df_temp = pd.read_csv(f, header=None, names=['raw'])
        df_temp['dt'] = file_name[:8]
        df_temp['hr'] = file_name[8:10]
        df_temp['user_id'] = df_temp['raw'].str.split(' ').str[0]
        df_temp['article_id'] = df_temp['raw'].str.split(' ').str[1:].str.join(' ').str.strip()
        read_df_lst.append(df_temp)

In [8]:
read = pd.concat(read_df_lst)

In [10]:
read.head()

,raw,dt,hr,user_id,article_id
0,#a055d0c3520e1c002531001928217887 @charlessay_...,20181121,14,#a055d0c3520e1c002531001928217887,@charlessay_30 @wal8am_27 @uglyduckmin_40 @ant...
1,#a9cba8bf098c149170315d4cd8d3082e @jinbread_88...,20181121,14,#a9cba8bf098c149170315d4cd8d3082e,@jinbread_88 @jinbread_87 @jinbread_89
2,#207c8cb590a843e247b9fe7fdd0e5281 @sangheeshyn...,20181121,14,#207c8cb590a843e247b9fe7fdd0e5281,@sangheeshyn_66
3,#d76887a5b30adbf793f742c6f67b57bd @tenbody_150...,20181121,14,#d76887a5b30adbf793f742c6f67b57bd,@tenbody_1506 @tenbody_1506
4,#b260d83247a4d9e3b22143c950f7f471 @maumdal_257...,20181121,14,#b260d83247a4d9e3b22143c950f7f471,@maumdal_257 @maumdal_257


In [25]:
def chainer(s):
    return list(chain.from_iterable(s.str.split(' ')))

In [17]:
read_cnt_by_user = read['article_id'].str.split(' ').map(len)

In [26]:
read_raw = pd.DataFrame({'dt': np.repeat(read['dt'], read_cnt_by_user),
                         'hr': np.repeat(read['hr'], read_cnt_by_user),
                         'user_id': np.repeat(read['user_id'], read_cnt_by_user),
                         'article_id': chainer(read['article_id'])})

In [32]:
read_raw.head()

,dt,hr,user_id,article_id
0,20181121,14,#a055d0c3520e1c002531001928217887,@charlessay_30
0,20181121,14,#a055d0c3520e1c002531001928217887,@wal8am_27
0,20181121,14,#a055d0c3520e1c002531001928217887,@uglyduckmin_40
0,20181121,14,#a055d0c3520e1c002531001928217887,@anti-essay_133
0,20181121,14,#a055d0c3520e1c002531001928217887,@roysday_125


In [50]:
train = read_raw[read_raw.dt <= '2019022']
dev = read_raw[read_raw.dt > '2019022']

In [52]:
print(len(train))
print(len(dev))

20624144
1486562


In [55]:
dev.head()

,dt,hr,user_id,article_id
0,20190222,19,#1f756fba1b5058dcedc2d70e630de726,@5bro_46
1,20190222,19,#77a2b884d3426d9e003b36c52035f742,@brunch_151
2,20190222,19,#8d494c4184dc92456379609b6f65d36b,@ubudsister_3
3,20190222,19,#4b354a788313de64d975718d3236faa6,@sunsutu_990
3,20190222,19,#4b354a788313de64d975718d3236faa6,@sunsutu_990


In [56]:
train = train[['user_id', 'article_id']].drop_duplicates()
dev = dev[['user_id', 'article_id']].drop_duplicates()

In [57]:
print(len(train))
print(len(dev))

11795765
854453


In [58]:
train.to_csv(conf.data + 'train_read.csv', index=False)
dev.to_csv(conf.data + 'dev_read.csv', index=False)